In [1]:
import pandas as pd
import numpy as np
from mido import Message, MidiFile, MidiTrack, MetaMessage
from collections import namedtuple

## Data processing: 

In [24]:
df = pd.read_csv('https://opendata.ecdc.europa.eu/covid19/casedistribution/csv')

df['dateRep'] = pd.to_datetime(df['dateRep'], format = '%d/%m/%Y')

continent_names_df = pd.read_csv(
    '~/Dropbox (BBC)/Visual Journalism/Data/2020/vjdata.26866.global.covid.interactive/\
derived/bbc_world_v2_4326_centroid_region.csv'
)

continent_df = pd.merge(df, 
         continent_names_df[['iso3', 'bbc-region-code']], 
         left_on = 'countryterritoryCode', 
         right_on = 'iso3').groupby(['dateRep', 'bbc-region-code'])[['cases', 'deaths']].sum().reset_index()

def rename_continent(bbc_region): 
    switcher = {
        'region_as': 'Asia',
        'region_af': 'Africa',
        'region_am_na': 'North America',
        'region_am_lac': 'Latin America',
        'region_me': 'Middle East',
        'region_eu': 'Europe',
        'region_oc': 'Oceania'
    }
    return switcher[bbc_region]

continent_df['continent_name'] = continent_df['bbc-region-code'].map(rename_continent)
continent_df = continent_df[['dateRep', 'continent_name', 'cases', 'deaths']]
continent_df.rename(columns = {'dateRep': 'date'}, inplace = True)

continent_df = continent_df.sort_values(by = ['continent_name', 'date']).reset_index(drop=True)
continent_df['cases_rolling_average'] = continent_df.groupby(
    'continent_name'
).rolling(7)['cases'].mean().reset_index(drop=True)
continent_df['deaths_rolling_average'] = continent_df.groupby(
    'continent_name'
).rolling(7)['deaths'].mean().reset_index(drop=True)
continent_df['cumulative_deaths'] = continent_df.groupby(['continent_name'])['deaths'].cumsum()

Produces two dataframes: continent_df, which is daily data aggregated by continent name, and weekly_continent_df, which is the same data aggregated by week and by continent name. Current plan is to use the latter. 

In [122]:
continent_df

,date,continent_name,cases,deaths,cases_rolling_average,deaths_rolling_average,cumulative_deaths
0,2019-12-31,Africa,0,0,NaN,NaN,0
1,2020-01-01,Africa,0,0,NaN,NaN,0
2,2020-01-02,Africa,0,0,NaN,NaN,0
3,2020-01-03,Africa,0,0,NaN,NaN,0
4,2020-01-04,Africa,0,0,NaN,NaN,0
...,...,...,...,...,...,...,...
2438,2020-12-09,Oceania,266,7,175.285714,1.714286,1143
2439,2020-12-10,Oceania,19,0,143.857143,1.571429,1143
2440,2020-12-11,Oceania,227,7,148.000000,2.428571,1150
2441,2020-12-12,Oceania,169,3,140.857143,2.571429,1153


In [128]:
continent_df.groupby('continent_name').describe()

cases                                                     \
                count          mean           std  min      25%      50%   
continent_name                                                             
Africa          349.0   6425.776504   5836.623530  0.0    426.0   6608.0   
Asia            349.0  37547.435530  35406.864704  0.0   2494.0  27183.0   
Europe          349.0  60701.366762  86512.504770  0.0  15047.0  22818.0   
Latin America   349.0  40442.931232  33671.778782  0.0   1521.0  44342.0   
Middle East     349.0  10697.275072   8521.723450  0.0   3002.0  11685.0   
North America   349.0  47340.693410  52872.593828  0.0  17431.0  34548.0   
Oceania         349.0    153.031519    203.881548  0.0     11.0     62.0   

                                  deaths               ...  \
                    75%       max  count         mean  ...   
continent_name                                         ...   
Africa          10784.0   19991.0  349.0   140.656160  ...   
Asia            66041.0  111675.0  349.0   588.229226  ...   
Europe          50184.0  322155.0  349.0  1364.833811  ...   
Latin America   70027.0  107910.0  349.0  1351.851003  ...   
Middle East     15341.0   42401.0  349.0   259.888252  ...   
North America   57525.0  241405.0  349.0   891.653295  ...   
Oceania           246.0    1526.0  349.0     3.303725  ...   

               deaths_rolling_average              cumulative_deaths  \
                                  75%          max             count   
continent_name                                                         
Africa                     238.500000   406.428571             349.0   
Asia                      1016.785714  1465.428571             349.0   
Europe                    1987.642857  5290.428571             349.0   
Latin America             2276.285714  3152.000000             349.0   
Middle East                411.142857   725.857143             349.0   
North America             1147.428571  2848.142857             349.0   
Oceania                      3.285714    22.571429             349.0   

                                                                      \
                         mean            std  min      25%       50%   
continent_name                                                         
Africa           13904.034384   15609.888397  0.0     63.0    5918.0   
Asia             62643.111748   68600.545810  0.0   3666.0   29327.0   
Europe          161987.670487  120896.908660  0.0  16842.0  190178.0   
Latin America   158485.352436  164891.301471  0.0    184.0   95941.0   
Middle East      26912.000000   27336.756442  0.0   2315.0   15718.0   
North America   121298.816619   97136.449002  0.0   1335.0  128405.0   
Oceania            381.320917     430.340663  0.0     14.0     131.0   

                                    
                     75%       max  
continent_name                      
Africa           27550.0   49089.0  
Asia            120544.0  205292.0  
Europe          223376.0  476327.0  
Latin America   316423.0  471796.0  
Middle East      46619.0   90701.0  
North America   205995.0  311187.0  
Oceania            890.0    1153.0  

[7 rows x 40 columns]

In [78]:
x = [1,4,5,6]
x[-1:]

[6]

In [127]:
df

,dateRep,day,month,year,cases,deaths,countriesAndTerritories,geoId,countryterritoryCode,popData2019,continentExp,Cumulative_number_for_14_days_of_COVID-19_cases_per_100000
0,2020-12-13,13,12,2020,298,9,Afghanistan,AF,AFG,38041757.0,Asia,7.052776
1,2020-12-12,12,12,2020,113,11,Afghanistan,AF,AFG,38041757.0,Asia,6.868768
2,2020-12-11,11,12,2020,63,10,Afghanistan,AF,AFG,38041757.0,Asia,7.134266
3,2020-12-10,10,12,2020,202,16,Afghanistan,AF,AFG,38041757.0,Asia,6.968658
4,2020-12-09,9,12,2020,135,13,Afghanistan,AF,AFG,38041757.0,Asia,6.963401
...,...,...,...,...,...,...,...,...,...,...,...,...
61682,2020-03-25,25,3,2020,0,0,Zimbabwe,ZW,ZWE,14645473.0,Africa,NaN
61683,2020-03-24,24,3,2020,0,1,Zimbabwe,ZW,ZWE,14645473.0,Africa,NaN
61684,2020-03-23,23,3,2020,0,0,Zimbabwe,ZW,ZWE,14645473.0,Africa,NaN
61685,2020-03-22,22,3,2020,1,0,Zimbabwe,ZW,ZWE,14645473.0,Africa,NaN


In [125]:
# The maximum cases value will be used to calibrate the volume of each track:
max_global_rolling_avg_deaths_value = continent_df['deaths_rolling_average'].max()

# 41 48 55 62 55 48
# Reference for instrument values, general midi: https://en.wikipedia.org/wiki/General_MIDI
Continent = namedtuple('Continent', ['name',          'pattern','octave_shift', 'pan', 'instrument', 'channel'])
asia =                  Continent(   'Asia',          [2,1,4],   -3,             120,   42,           0) #cello
middle_east =           Continent(   'Middle East',   [4,3,2],   -1,             100,   73,           1) #flute
africa =                Continent(   'Africa',        [1,4,3],   -2,              80,   41,           2) #viola
europe =                Continent(   'Europe',        [2,1,4],   -1,              63,   71,           3) #clnet
latin_america =         Continent(   'Latin America', [3,2,1],   -1,              40,   40,           4) #vln
north_america =         Continent(   'North America', [4,3,2],   -1,              10,   40,           5) #vln

continents = [asia, middle_east, africa, europe, latin_america, north_america]

class Phrase(): 
    def __init__(self, chord, volume, pattern, note_count, octave_shift, is_first_day, offset): 
        self.chord = chord
        self.volume = volume
        self.pattern = pattern
        self.note_count = note_count
        self.octave_shift = octave_shift
        self.is_first_day = is_first_day
        self.offset = offset
        self.notes = self.define_notes()
        self.beat_duration = 240
        
    def define_notes(self): 
        if self.chord == 'i7': 
            return [62, 60, 65, 69]
        elif self.chord == 'iiø': 
            return [64, 58, 64, 67]
        elif self.chord == 'V64': 
            return [64, 61, 64, 67]
        elif self.chord == 'vi64': 
            return [65, 58, 62, 65]
        elif self.chord == 'i6': 
            return [65, 57, 62, 65]
        elif self.chord == 'iv': 
            return [67, 58, 62, 65]
        elif self.chord == 'N': 
            return [63, 58, 63, 67]
        
    def add_to_track(self, track):
        main_note = self.notes[self.pattern[1] - 1] + self.octave_shift * 12
        deputy_note = self.notes[self.pattern[0] - 1] + self.octave_shift * 12 
        outlier_note = self.notes[self.pattern[2] - 1] + self.octave_shift * 12 
#         notes_to_play = [main_note] * self.note_count
        
        if self.note_count == 1: 
            notes_to_play = [main_note]
        elif self.note_count == 2: 
            notes_to_play = [deputy_note, main_note]
        elif self.note_count == 3: 
            notes_to_play = [main_note, deputy_note, main_note]
        elif self.note_count == 4: 
            notes_to_play = [deputy_note, main_note, deputy_note, main_note]
        elif self.note_count == 5: 
            notes_to_play = [deputy_note, main_note, deputy_note, main_note, deputy_note]
        elif self.note_count == 6: 
            notes_to_play = [deputy_note, main_note, deputy_note, main_note, outlier_note, deputy_note]
        
        if self.is_first_day: 
            delay = self.offset * self.beat_duration
        else: 
            delay = int(self.beat_duration * 0.5)
            
        note = notes_to_play.pop(0)
        message = Message('note_on', note = note, time = delay)
        track.append(message)      
        min_volume = self.volume
        max_volume = self.volume + 40
        for volume in range(min_volume, max_volume, 4): 
            if self.volume == 0: 
                volume = 0
            track.append(Message('control_change', control=7, value=volume, time = 12))
        
        if len(notes_to_play) > 0: 
            message = Message('note_off', note = note, time = 0)
            track.append(message)
            note = notes_to_play.pop(0)
            message = Message('note_on', note = note, time = 0)
            track.append(message)
        
        for volume in range(min_volume, max_volume, 4)[::-1]: 
            if self.volume == 0: 
                volume = 0
            track.append(Message('control_change', control=7, value=volume, time = 12))
        
        if len(notes_to_play) > 0: 
            message = Message('note_off', note = note, time = 0)
            track.append(message)
            note = notes_to_play.pop(0)
            message = Message('note_on', note = note, time = 0)
            track.append(message)
        
        if len(notes_to_play) > 0: 
            message = Message('note_off', note = note, time = int(self.beat_duration))
            track.append(message)
            note = notes_to_play.pop(0)
            message = Message('note_on', note = note, time = 0)
            track.append(message)
        
        if self.note_count <= 4: 
            fade_at_n_beats = 4
            if self.note_count == 4: 
                fade_at_n_beats = 3
            fade_at_t = int(self.beat_duration * fade_at_n_beats)
            track.append(Message('control_change', control=7, value=self.volume, time = fade_at_t))
            for volume in range(min_volume - 40, min_volume, 4)[::-1]: 
                if volume < 0: 
                    volume = 0
                track.append(Message('control_change', control=7, value=volume, time = 12))

            message = Message('note_off', note = note, velocity = 0, time = 0)
            track.append(message)

            track.append(Message('control_change', control=7, value=0, time=0))
            
            
        elif self.note_count == 5:
            message = Message('note_off', note = note, time = int(self.beat_duration * 3))
            track.append(message)
            note = notes_to_play.pop(0)
            message = Message('note_on', note = note, time = 0)
            track.append(message)
            message = Message('note_off', note = note, time = int(self.beat_duration * 0.5))
            track.append(message)
            
            
        elif self.note_count == 6:
            message = Message('note_off', note = note, time = int(self.beat_duration * 2.5))
            track.append(message)
            note = notes_to_play.pop(0)
            message = Message('note_on', note = note, time = 0)
            track.append(message)
            message = Message('note_off', note = note, time = int(self.beat_duration * 0.5))
            track.append(message)
            note = notes_to_play.pop(0)
            message = Message('note_on', note = note, time = 0)
            track.append(message)
            message = Message('note_off', note = note, time = int(self.beat_duration * 0.5))
            track.append(message)

# chords = ['i7', 'iiø', 'V64', 'vi64', 'i6', 'iv', 'N']    
chords = ['iiø', 'iiø', 'iiø', 'iiø', 'iiø', 'iiø', 'V64']    

mid = MidiFile()
for continent in continents:
    name = continent.name
    pattern = continent.pattern
    channel = continent.channel
    
    print(continent)
    filtered_df = continent_df.loc[continent_df['continent_name'] == continent.name]
    rolling_avg_deaths_data = filtered_df['deaths_rolling_average'].apply(lambda x: 0 if np.isnan(x) else int(x))
    cumulative_deaths_data = filtered_df['cumulative_deaths'].apply(lambda x: 0 if np.isnan(x) else int(x))
#     deaths_data = continent_df.groupby('date')['cumulative_deaths'].sum().values
    track = MidiTrack()
    mid.tracks.append(track)
    # Set up track:
    track.append(MetaMessage('track_name', name=name, time=0))
    track.append(MetaMessage('time_signature', numerator=6, denominator=8, time=0)) # does not work
    # Instrumentation:
    track.append(Message('program_change', channel=channel, program=continent.instrument, time=0))
    # Pan:
    track.append(Message('control_change', channel=channel, control=10, value=continent.pan, time=0))
    
    max_continent_rolling_avg_deaths_value = rolling_avg_deaths_data.max()
    rolling_avg_threshold_width = max_continent_rolling_avg_deaths_value / 6
    
    max_cumulative_deaths_value = cumulative_deaths_data.max()
    cumulative_deaths_threshold_width = max_cumulative_deaths_value / 3
    
    counter = 0
    for rolling_avg_deaths, cumulative_deaths in zip(rolling_avg_deaths_data, cumulative_deaths_data): 
        print(rolling_avg_deaths, cumulative_deaths, cumulative_deaths_threshold_width)
        if rolling_avg_deaths > 0: 
            volume = int(rolling_avg_deaths / max_global_rolling_avg_deaths_value * 80) + 6
        else: 
            volume = 0
        chord = chords[counter % 7]
        is_first_day = counter == 0
        counter += 1
        
        for i in range(1, 7):
            if rolling_avg_deaths <= rolling_avg_threshold_width * i: 
                note_count = i
        
        octave_shift = continent.octave_shift
        for i in range(3):
            if cumulative_deaths >= cumulative_deaths_threshold_width * i: 
                octave_shift = continent.octave_shift + i
                
        
        phrase = Phrase(chord, volume, pattern, note_count, octave_shift, is_first_day, channel)
        phrase.add_to_track(track)
        
        
    
mid.save('./output/v6_test_009.mid')

Continent(name='Asia', pattern=[2, 1, 4], octave_shift=-3, pan=120, instrument=42, channel=0)
0 0 68430.66666666667
-3
0 0 68430.66666666667
-3
0 0 68430.66666666667
-3
0 0 68430.66666666667
-3
0 0 68430.66666666667
-3
0 0 68430.66666666667
-3
0 0 68430.66666666667
-3
0 0 68430.66666666667
-3
0 0 68430.66666666667
-3
0 0 68430.66666666667
-3
0 0 68430.66666666667
-3
0 1 68430.66666666667
-3
0 1 68430.66666666667
-3
0 1 68430.66666666667
-3
0 1 68430.66666666667
-3
0 2 68430.66666666667
-3
0 2 68430.66666666667
-3
0 2 68430.66666666667
-3
0 2 68430.66666666667
-3
0 3 68430.66666666667
-3
0 3 68430.66666666667
-3
0 6 68430.66666666667
-3
2 17 68430.66666666667
-3
2 17 68430.66666666667
-3
3 26 68430.66666666667
-3
5 41 68430.66666666667
-3
7 56 68430.66666666667
-3
11 81 68430.66666666667
-3
14 106 68430.66666666667
-3
16 132 68430.66666666667
-3
21 170 68430.66666666667
-3
26 213 68430.66666666667
-3
31 259 68430.66666666667
-3
35 305 68430.66666666667
-3
40 362 68430.66666666667
-3
45 

1168 153991 68430.66666666667
-1
1131 154976 68430.66666666667
-1
1094 155965 68430.66666666667
-1
1072 157114 68430.66666666667
-1
1054 158184 68430.66666666667
-1
1070 159487 68430.66666666667
-1
1030 160292 68430.66666666667
-1
1016 161105 68430.66666666667
-1
1016 162090 68430.66666666667
-1
1017 163086 68430.66666666667
-1
988 164035 68430.66666666667
-1
980 165044 68430.66666666667
-1
911 165864 68430.66666666667
-1
895 166563 68430.66666666667
-1
888 167321 68430.66666666667
-1
854 168070 68430.66666666667
-1
822 168841 68430.66666666667
-1
800 169638 68430.66666666667
-1
764 170398 68430.66666666667
-1
746 171087 68430.66666666667
-1
744 171771 68430.66666666667
-1
744 172534 68430.66666666667
-1
749 173314 68430.66666666667
-1
781 174312 68430.66666666667
-1
798 175230 68430.66666666667
-1
806 176044 68430.66666666667
-1
819 176825 68430.66666666667
-1
824 177541 68430.66666666667
-1
823 178299 68430.66666666667
-1
813 179010 68430.66666666667
-1
791 179855 68430.66666666667
-

655 80721 30233.666666666668
1
591 80931 30233.666666666668
1
658 82067 30233.666666666668
1
725 82738 30233.666666666668
1
643 83297 30233.666666666668
1
634 83836 30233.666666666668
1
622 84450 30233.666666666668
1
609 84984 30233.666666666668
1
659 85550 30233.666666666668
1
575 86092 30233.666666666668
1
553 86614 30233.666666666668
1
544 87108 30233.666666666668
1
534 87574 30233.666666666668
1
511 88032 30233.666666666668
1
502 88499 30233.666666666668
1
490 88984 30233.666666666668
1
479 89449 30233.666666666668
1
472 89922 30233.666666666668
1
457 90307 30233.666666666668
1
446 90701 30233.666666666668
1
Continent(name='Africa', pattern=[1, 4, 3], octave_shift=-2, pan=80, instrument=41, channel=2)
0 0 16363.0
-2
0 0 16363.0
-2
0 0 16363.0
-2
0 0 16363.0
-2
0 0 16363.0
-2
0 0 16363.0
-2
0 0 16363.0
-2
0 0 16363.0
-2
0 0 16363.0
-2
0 0 16363.0
-2
0 0 16363.0
-2
0 0 16363.0
-2
0 0 16363.0
-2
0 0 16363.0
-2
0 0 16363.0
-2
0 0 16363.0
-2
0 0 16363.0
-2
0 0 16363.0
-2
0 0 16363.0
-2


0
435 220606 158775.66666666666
0
423 220939 158775.66666666666
0
443 221406 158775.66666666666
0
453 221912 158775.66666666666
0
501 222774 158775.66666666666
0
518 223376 158775.66666666666
0
535 223978 158775.66666666666
0
572 224615 158775.66666666666
0
592 225085 158775.66666666666
0
599 225600 158775.66666666666
0
618 226240 158775.66666666666
0
600 226978 158775.66666666666
0
614 227674 158775.66666666666
0
629 228383 158775.66666666666
0
631 229032 158775.66666666666
0
653 229656 158775.66666666666
0
666 230264 158775.66666666666
0
669 230925 158775.66666666666
0
690 231808 158775.66666666666
0
713 232665 158775.66666666666
0
723 233444 158775.66666666666
0
739 234210 158775.66666666666
0
745 234876 158775.66666666666
0
737 235423 158775.66666666666
0
775 236352 158775.66666666666
0
787 237318 158775.66666666666
0
811 238343 158775.66666666666
0
853 239418 158775.66666666666
0
856 240206 158775.66666666666
0
899 241173 158775.66666666666
0
951 242080 158775.66666666666
0
937 24

0 0 103729.0
-1
0 0 103729.0
-1
0 0 103729.0
-1
0 0 103729.0
-1
0 0 103729.0
-1
0 0 103729.0
-1
0 0 103729.0
-1
0 0 103729.0
-1
0 0 103729.0
-1
0 0 103729.0
-1
0 0 103729.0
-1
0 0 103729.0
-1
0 0 103729.0
-1
0 0 103729.0
-1
0 0 103729.0
-1
0 0 103729.0
-1
0 0 103729.0
-1
0 0 103729.0
-1
0 0 103729.0
-1
0 0 103729.0
-1
0 0 103729.0
-1
0 0 103729.0
-1
0 0 103729.0
-1
0 0 103729.0
-1
0 0 103729.0
-1
0 0 103729.0
-1
0 0 103729.0
-1
0 1 103729.0
-1
0 2 103729.0
-1
0 6 103729.0
-1
1 9 103729.0
-1
1 11 103729.0
-1
1 12 103729.0
-1
2 14 103729.0
-1
2 17 103729.0
-1
2 21 103729.0
-1
3 27 103729.0
-1
2 29 103729.0
-1
2 31 103729.0
-1
4 41 103729.0
-1
4 48 103729.0
-1
5 58 103729.0
-1
7 70 103729.0
-1
8 89 103729.0
-1
12 116 103729.0
-1
18 159 103729.0
-1
17 160 103729.0
-1
32 272 103729.0
-1
42 358 103729.0
-1
60 491 103729.0
-1
75 614 103729.0
-1
101 828 103729.0
-1
132 1085 103729.0
-1
167 1335 103729.0
-1
212 1760 103729.0
-1
270 2251 103729.0
-1
297 2570 103729.0
-1
377 3259 103729.0
-1
478 

In [57]:
for msg in mid.tracks[0]:
    print(msg)

<meta message track_name name='Asia' time=0>
<meta message time_signature numerator=6 denominator=8 clocks_per_click=24 notated_32nd_notes_per_beat=8 time=0>
program_change channel=0 program=42 time=0
control_change channel=0 control=10 value=120 time=0
note_on channel=0 note=50 velocity=64 time=0
control_change channel=0 control=7 value=0 time=12
control_change channel=0 control=7 value=0 time=12
control_change channel=0 control=7 value=0 time=12
control_change channel=0 control=7 value=0 time=12
control_change channel=0 control=7 value=0 time=12
control_change channel=0 control=7 value=0 time=12
control_change channel=0 control=7 value=0 time=12
control_change channel=0 control=7 value=0 time=12
control_change channel=0 control=7 value=0 time=12
control_change channel=0 control=7 value=0 time=12
control_change channel=0 control=7 value=0 time=12
control_change channel=0 control=7 value=0 time=12
control_change channel=0 control=7 value=0 time=12
control_change channel=0 control=7 val

control_change channel=0 control=7 value=0 time=12
control_change channel=0 control=7 value=0 time=12
control_change channel=0 control=7 value=0 time=12
control_change channel=0 control=7 value=0 time=12
control_change channel=0 control=7 value=0 time=12
note_off channel=0 note=55 velocity=0 time=0
control_change channel=0 control=7 value=0 time=0
note_on channel=0 note=51 velocity=64 time=0
control_change channel=0 control=7 value=6 time=12
control_change channel=0 control=7 value=10 time=12
control_change channel=0 control=7 value=14 time=12
control_change channel=0 control=7 value=18 time=12
control_change channel=0 control=7 value=22 time=12
control_change channel=0 control=7 value=26 time=12
control_change channel=0 control=7 value=30 time=12
control_change channel=0 control=7 value=34 time=12
control_change channel=0 control=7 value=38 time=12
control_change channel=0 control=7 value=42 time=12
control_change channel=0 control=7 value=42 time=12
control_change channel=0 control=7

control_change channel=0 control=7 value=0 time=12
note_off channel=0 note=50 velocity=0 time=0
control_change channel=0 control=7 value=0 time=0
note_on channel=0 note=52 velocity=64 time=0
control_change channel=0 control=7 value=6 time=12
control_change channel=0 control=7 value=10 time=12
control_change channel=0 control=7 value=14 time=12
control_change channel=0 control=7 value=18 time=12
control_change channel=0 control=7 value=22 time=12
control_change channel=0 control=7 value=26 time=12
control_change channel=0 control=7 value=30 time=12
control_change channel=0 control=7 value=34 time=12
control_change channel=0 control=7 value=38 time=12
control_change channel=0 control=7 value=42 time=12
control_change channel=0 control=7 value=42 time=12
control_change channel=0 control=7 value=38 time=12
control_change channel=0 control=7 value=34 time=12
control_change channel=0 control=7 value=30 time=12
control_change channel=0 control=7 value=26 time=12
control_change channel=0 contr

note_on channel=0 note=53 velocity=64 time=0
control_change channel=0 control=7 value=9 time=12
control_change channel=0 control=7 value=13 time=12
control_change channel=0 control=7 value=17 time=12
control_change channel=0 control=7 value=21 time=12
control_change channel=0 control=7 value=25 time=12
control_change channel=0 control=7 value=29 time=12
control_change channel=0 control=7 value=33 time=12
control_change channel=0 control=7 value=37 time=12
control_change channel=0 control=7 value=41 time=12
control_change channel=0 control=7 value=45 time=12
control_change channel=0 control=7 value=45 time=12
control_change channel=0 control=7 value=41 time=12
control_change channel=0 control=7 value=37 time=12
control_change channel=0 control=7 value=33 time=12
control_change channel=0 control=7 value=29 time=12
control_change channel=0 control=7 value=25 time=12
control_change channel=0 control=7 value=21 time=12
control_change channel=0 control=7 value=17 time=12
control_change chann

control_change channel=0 control=7 value=27 time=12
control_change channel=0 control=7 value=31 time=12
control_change channel=0 control=7 value=35 time=12
control_change channel=0 control=7 value=39 time=12
control_change channel=0 control=7 value=43 time=12
control_change channel=0 control=7 value=47 time=12
control_change channel=0 control=7 value=51 time=12
control_change channel=0 control=7 value=51 time=12
control_change channel=0 control=7 value=47 time=12
control_change channel=0 control=7 value=43 time=12
control_change channel=0 control=7 value=39 time=12
control_change channel=0 control=7 value=35 time=12
control_change channel=0 control=7 value=31 time=12
control_change channel=0 control=7 value=27 time=12
control_change channel=0 control=7 value=23 time=12
control_change channel=0 control=7 value=19 time=12
control_change channel=0 control=7 value=15 time=12
control_change channel=0 control=7 value=15 time=1080
control_change channel=0 control=7 value=11 time=12
control_ch

control_change channel=0 control=7 value=50 time=12
control_change channel=0 control=7 value=54 time=12
control_change channel=0 control=7 value=58 time=12
control_change channel=0 control=7 value=62 time=12
control_change channel=0 control=7 value=62 time=12
control_change channel=0 control=7 value=58 time=12
control_change channel=0 control=7 value=54 time=12
control_change channel=0 control=7 value=50 time=12
control_change channel=0 control=7 value=46 time=12
control_change channel=0 control=7 value=42 time=12
control_change channel=0 control=7 value=38 time=12
control_change channel=0 control=7 value=34 time=12
control_change channel=0 control=7 value=30 time=12
control_change channel=0 control=7 value=26 time=12
control_change channel=0 control=7 value=26 time=1080
control_change channel=0 control=7 value=22 time=12
control_change channel=0 control=7 value=18 time=12
control_change channel=0 control=7 value=14 time=12
control_change channel=0 control=7 value=10 time=12
control_ch

control_change channel=0 control=7 value=68 time=12
control_change channel=0 control=7 value=64 time=12
control_change channel=0 control=7 value=60 time=12
control_change channel=0 control=7 value=56 time=12
control_change channel=0 control=7 value=52 time=12
control_change channel=0 control=7 value=48 time=12
control_change channel=0 control=7 value=44 time=12
control_change channel=0 control=7 value=40 time=12
control_change channel=0 control=7 value=36 time=12
control_change channel=0 control=7 value=32 time=12
control_change channel=0 control=7 value=32 time=1080
control_change channel=0 control=7 value=28 time=12
control_change channel=0 control=7 value=24 time=12
control_change channel=0 control=7 value=20 time=12
control_change channel=0 control=7 value=16 time=12
control_change channel=0 control=7 value=12 time=12
control_change channel=0 control=7 value=8 time=12
control_change channel=0 control=7 value=4 time=12
control_change channel=0 control=7 value=0 time=12
control_chang

control_change channel=0 control=7 value=46 time=12
control_change channel=0 control=7 value=42 time=12
control_change channel=0 control=7 value=38 time=12
control_change channel=0 control=7 value=34 time=12
control_change channel=0 control=7 value=30 time=12
control_change channel=0 control=7 value=26 time=12
control_change channel=0 control=7 value=22 time=12
control_change channel=0 control=7 value=22 time=1080
control_change channel=0 control=7 value=18 time=12
control_change channel=0 control=7 value=14 time=12
control_change channel=0 control=7 value=10 time=12
control_change channel=0 control=7 value=6 time=12
control_change channel=0 control=7 value=2 time=12
control_change channel=0 control=7 value=0 time=12
control_change channel=0 control=7 value=0 time=12
control_change channel=0 control=7 value=0 time=12
control_change channel=0 control=7 value=0 time=12
control_change channel=0 control=7 value=0 time=12
note_off channel=0 note=53 velocity=0 time=0
control_change channel=0